In [1]:
import psycopg2
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
#import constants
from datetime import date, datetime, timedelta
load_dotenv()
pd.options.mode.chained_assignment = None

from dateutil.relativedelta import relativedelta


In [2]:
df = pd.read_excel(r"D:\VL1251\ETL\old_data\wh\order\202501~0419 ĐĐH.xlsx")

In [4]:
date_col = ['Ngày ĐĐH','Ngày CT','Ngày dự tính giao hàng ban đầu','Ngày dự định giao hàng']

In [6]:
for col in date_col:
    df[col] = pd.to_datetime(df[col], dayfirst=True)

In [10]:
df1 = df[df['Ngày dự định giao hàng']>='2025-04-20']

In [11]:
df_grouped = df1.groupby('KH Tên gọi tắt').agg({'SL đơn đặt':'sum'}).reset_index()

In [13]:
df_grouped.sort_values(by='SL đơn đặt', ascending=False)

,KH Tên gọi tắt,SL đơn đặt
13,大森 TIM BER,35951.0
16,太平洋 RETURN GOLD,21610.0
11,凱勝 KAISER 1,17405.0
5,TRENDEX,15875.0
30,永誠 VĨNH THÀNH,6800.0
1,DELTA,5030.0
8,上順 SANG SHUN,4950.0
6,"VFR CO.,LTD",4890.0
25,景楊 YOU JIE,4402.0
15,大顺 ĐẠI THUẬN,3565.0


In [2]:
import pytz
a = pytz.all_timezones
for i in a:
    if 'Asia' in i:
        print(i)

Asia/Aden
Asia/Almaty
Asia/Amman
Asia/Anadyr
Asia/Aqtau
Asia/Aqtobe
Asia/Ashgabat
Asia/Ashkhabad
Asia/Atyrau
Asia/Baghdad
Asia/Bahrain
Asia/Baku
Asia/Bangkok
Asia/Barnaul
Asia/Beirut
Asia/Bishkek
Asia/Brunei
Asia/Calcutta
Asia/Chita
Asia/Choibalsan
Asia/Chongqing
Asia/Chungking
Asia/Colombo
Asia/Dacca
Asia/Damascus
Asia/Dhaka
Asia/Dili
Asia/Dubai
Asia/Dushanbe
Asia/Famagusta
Asia/Gaza
Asia/Harbin
Asia/Hebron
Asia/Ho_Chi_Minh
Asia/Hong_Kong
Asia/Hovd
Asia/Irkutsk
Asia/Istanbul
Asia/Jakarta
Asia/Jayapura
Asia/Jerusalem
Asia/Kabul
Asia/Kamchatka
Asia/Karachi
Asia/Kashgar
Asia/Kathmandu
Asia/Katmandu
Asia/Khandyga
Asia/Kolkata
Asia/Krasnoyarsk
Asia/Kuala_Lumpur
Asia/Kuching
Asia/Kuwait
Asia/Macao
Asia/Macau
Asia/Magadan
Asia/Makassar
Asia/Manila
Asia/Muscat
Asia/Nicosia
Asia/Novokuznetsk
Asia/Novosibirsk
Asia/Omsk
Asia/Oral
Asia/Phnom_Penh
Asia/Pontianak
Asia/Pyongyang
Asia/Qatar
Asia/Qostanay
Asia/Qyzylorda
Asia/Rangoon
Asia/Riyadh
Asia/Saigon
Asia/Sakhalin
Asia/Samarkand
Asia/Seoul
Asia/

In [2]:
from utils.db import Database

In [4]:
Database.initialize(
    database=os.getenv('WAREHOUSE_NAME'),
    user=os.getenv('WAREHOUSE_USER'),
    password=os.getenv('WAREHOUSE_PASSWORD'),
    host=os.getenv('WAREHOUSE_HOST'),
    port=os.getenv('WAREHOUSE_PORT'),
    minconn=1,
    maxconn=20
)

In [11]:
factory_name='大森 TIM BER'

In [12]:
selected_year = 2025

In [13]:
from utils.query.wh.extract import get_mtd_by_month, get_factory_code

In [14]:
factory_code = get_factory_code(factory_name)
# Get sales
df_sales = get_mtd_by_month(selected_year, 1, 31, factory_code, "fact_sales", "sales_quantity", "sales_date")
df_sales.columns = ['month','sales_quantity','sales_quantity_timber']
df_sales.drop(columns = ['sales_quantity'], inplace=True)


In [15]:
df_sales

,month,sales_quantity_timber
0,1,25590.0


In [16]:

# Get planned deliveries
df_plan  = get_mtd_by_month(selected_year, 1, 31, factory_code, "fact_order", "order_quantity", "estimated_delivery_date")
df_plan.columns = ['month','plan_quantity','plan_quantity_timber']
df_plan.drop(columns = ['plan_quantity'], inplace=True)


In [17]:
df_plan

,month,plan_quantity_timber
0,1,40697.0
1,2,33976.0
2,9,0.0


In [18]:
conn = Database.get_connection()
cur = conn.cursor()

In [29]:
cur.execute("""DELETE from fact_order where order_code = '2201-240829022-0006'""")

In [30]:
conn.commit()

In [27]:
data = cur.fetchall()
column_names = [description[0] for description in cur.description]
df = pd.DataFrame(data=data, columns=column_names)

In [28]:
df

,order_date,order_code,ct_date,factory_code,factory_order_code,tax_type,department,salesman,deposit_rate,payment_registration_code,...,package_order_quantity,delivered_package_order_quantity,unit,package_unit,estimated_delivery_date,original_estimated_delivery_date,pre_ct,finish_code,import_timestamp,import_wh_timestamp
0,2024-08-29,2201-240829022-0006,2024-08-29,31095,DHM-24/08-091,Thuế cộng riêng,業務部 NV,陳國勇,0,2004.0,...,1.0,1.0,kg,T,2025-09-04,2024-08-30,2101-220401022-0099,T? ??ng k?t thuc,2024-12-03 11:30:23.803593,2024-12-03 13:07:25.288145


In [25]:
df.to_excel('temp.xlsx',index=False)

In [ ]:

# Merge for plotting
df = df_sales.merge(df_plan, on='month', how='outer')
df.columns = ['month','sales_quantity','plan_quantity']

df.fillna(0,inplace=True)

df['percentage'] = df.apply(

    lambda row: 100 if row['plan_quantity'] == 0 else 
                0 if row['sales_quantity'] == 0 else 
                (row['sales_quantity'] / row['plan_quantity'] * 100),
    axis=1
)

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
d1 = dict(Start= '2021-10-10 02:00:00', Finish = '2021-10-10 09:00:00', Task = 'Sleep')
d2 = dict(Start= '2021-10-10 09:00:00', Finish = '2021-10-10 09:30:00', Task = 'EAT')
d3 = dict(Start= '2021-10-10 09:30:00', Finish = '2021-10-10 12:00:00', Task = 'Study')
d4 = dict(Start= '2021-10-10 12:00:00', Finish = '2021-10-10 16:00:00', Task = 'Work')
d5 = dict(Start= '2021-10-10 16:00:00', Finish = '2021-10-10 16:50:00', Task = 'EAT')
d6 = dict(Start= '2021-10-10 17:00:00', Finish = '2021-10-10 20:00:00', Task = 'Study')
d8 = dict(Start= '2021-10-10 20:00:00', Finish = '2021-10-10 20:40:00', Task = 'EAT')
d7 = dict(Start= '2021-10-10 21:00:00', Finish = '2021-10-11 05:00:00', Task = 'Sleep')

df = pd.DataFrame([d1,d2,d3,d4,d5,d6,d7,d8])

In [7]:
gantt = px.timeline(df, x_start='Start', x_end = 'Finish', color = 'Task', height=150, y=['']*df.shape[0]) 
gantt.show()